# Personal Finance

Date: 9/1/2024

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# equity months: 2, 5, 8, 11
# bonus months: 5, 11
months_till_next_equity = 3
months_till_next_bonus = 3

car_principle = 31_835.52
min_student_loan_payment = 280.22

current_savings = 30_000
savings_target = 50_000

In [3]:
df_student = pd.DataFrame(data=[
    ['AA', 9_578.15, 3.150],
    ['AC', 4_826.64, 3.610],
    ['AD', 8_862.97, 3.610],
], columns=['group', 'principle', 'apr'])

## Budget

### Income

In [4]:
base_salary = 185_000

In [5]:
bonus_factor = 0.12 # standard base
annual_bonus = base_salary * bonus_factor
biannual_bonus = annual_bonus / 2
print(f'annual_bonus = ${annual_bonus}')
print(f'biannual_bonus = ${biannual_bonus}')

annual_bonus = $22200.0
biannual_bonus = $11100.0


In [6]:
share_price = 350.00
num_shares = 202
annual_equity = share_price * num_shares
quarterly_equity = annual_equity / 4
print(f'annual_equity = ${annual_equity}')
print(f'quarterly_equity = ${quarterly_equity}')

annual_equity = $70700.0
quarterly_equity = $17675.0


In [7]:
tax_rate = 0.34
bonus_tax_rate = 0.42

In [8]:
other_monthly_income = 0 # only 1-2 more payments (TBD on amount and when)

In [9]:
annual_compensation = base_salary + annual_equity + annual_bonus
print(f'annual_compensation = ${annual_compensation}')

annual_compensation = $277900.0


In [10]:
total_annual_income = annual_compensation + 8 * other_monthly_income # for 2024
print(f'total_annual_income = ${total_annual_income}')

total_annual_income = $277900.0


In [11]:
net_annual_income = base_salary * (1 - tax_rate) + (annual_equity + annual_bonus) * (1 - bonus_tax_rate) + 8 * other_monthly_income
print(f'net_annual_income = ${net_annual_income}')

net_annual_income = $175982.0


In [12]:
monthly_salary = base_salary / 12
print(f'monthly_salary = ${round(monthly_salary, 2)}')

monthly_salary = $15416.67


In [13]:
total_monthly_income = base_salary / 12 + other_monthly_income
print(f'total_monthly_income = ${round(total_monthly_income, 2)}')

total_monthly_income = $15416.67


In [14]:
net_monthly_income = base_salary * (1 - tax_rate) / 12 + other_monthly_income
print(f'net_monthly_income = ${round(net_monthly_income, 2)}')

net_monthly_income = $10175.0


In [15]:
total_per_paycheck = base_salary / 12 / 2
print(f'total_per_paycheck = ${round(total_per_paycheck, 2)}')

total_per_paycheck = $7708.33


In [16]:
net_per_paycheck = total_per_paycheck * (1 - tax_rate)
print(f'net_per_paycheck = ${round(net_per_paycheck, 2)}')

net_per_paycheck = $5087.5


In [17]:
net_quarterly_equity = quarterly_equity * (1 - bonus_tax_rate)
print(f'net_quarterly_equity = ${round(net_quarterly_equity, 2)}')

net_quarterly_equity = $10251.5


In [18]:
net_biannual_bonus = biannual_bonus * (1 - bonus_tax_rate)
print(f'net_biannual_bonus = ${round(net_biannual_bonus, 2)}')

net_biannual_bonus = $6438.0


In [19]:
effective_total_monthly_income = total_monthly_income + annual_bonus / 12 + annual_equity / 12
print(f'effective_total_monthly_income = ${round(effective_total_monthly_income, 2)}')

effective_total_monthly_income = $23158.33


In [20]:
effective_net_monthly_income = net_annual_income / 12
print(f'effective_net_monthly_income = ${round(effective_net_monthly_income, 2)}')

effective_net_monthly_income = $14665.17


### Costs

In [21]:
fixed_costs = sum([
    3000, # rent
    170, # utilities
    50, # health insurance
    120, # car insurance
    15 * 2 + 12 * 2, # subscriptions
    389, # student loans
    1249, # car loan
])

print(f'fixed_costs = ${fixed_costs}')

fixed_costs = $5032


In [22]:
variable_costs_sc = sum([
    30 * 30, # food + coffee + alcohol
    100, # gas
    450 + 150, # travel (GSP-SFO airfare + 1 hotel)
    200, # misc
])

print(f'variable_costs_sc = ${variable_costs_sc}')

variable_costs_sc = $1800


In [23]:
variable_costs_ca = sum([
    50 * 30, # food + coffee + alcohol
    300, # gas
    450 + 4 * 100, # travel (GSP-SFO airfare + 4 hotels)
    200, # misc
])

print(f'variable_costs_ca = ${variable_costs_ca}')

variable_costs_ca = $2850


In [24]:
total_monthly_costs_sc = fixed_costs + variable_costs_sc
print(f'total_monthly_costs_sc = ${total_monthly_costs_sc}')

total_monthly_costs_sc = $6832


In [25]:
total_monthly_costs_ca = fixed_costs + variable_costs_ca
print(f'total_monthly_costs_ca = ${total_monthly_costs_ca}')

total_monthly_costs_ca = $7882


In [26]:
pct_time_in_sc = 0.5
avg_total_monthly_costs = pct_time_in_sc * total_monthly_costs_sc + (1 - pct_time_in_sc) * total_monthly_costs_ca
print(f'avg_total_monthly_costs = ${avg_total_monthly_costs}')

avg_total_monthly_costs = $7357.0


In [27]:
total_annual_costs = total_monthly_costs_sc * 6 + total_monthly_costs_ca * 6
print(f'total_annual_costs = ${total_annual_costs}')

total_annual_costs = $88284


### Profits

In [28]:
avg_monthly_profit = net_monthly_income - avg_total_monthly_costs
print(f'avg_monthly_profit = ${round(avg_monthly_profit, 2)}')

avg_monthly_profit = $2818.0


In [29]:
avg_effective_monthly_profit = effective_net_monthly_income - avg_total_monthly_costs
print(f'avg_effective_monthly_profit = ${round(avg_effective_monthly_profit, 2)}')

avg_effective_monthly_profit = $7308.17


In [30]:
annual_profit = net_annual_income - total_annual_costs
print(f'annual_profit = ${annual_profit}')

annual_profit = $87698.0


## Student Loan

In [31]:
df_student['monthly_interest'] = (df_student['principle'] * df_student['apr'] / 12 / 100).round(2)
df_student

,group,principle,apr,monthly_interest
0,AA,9578.15,3.15,25.14
1,AC,4826.64,3.61,14.52
2,AD,8862.97,3.61,26.66


In [32]:
effective_student_apr = np.dot(df_student['principle'] / df_student['principle'].sum(), df_student['apr'])
print(f'effective_student_apr = {round(effective_student_apr, 3)}%')

effective_student_apr = 3.421%


In [33]:
total_student_principle = df_student['principle'].sum()
print(f'total_student_principle = ${round(total_student_principle, 2)}')

total_student_principle = $23267.76


In [34]:
def amortization_schedule(df, min_payment, extra_payments=[], start_at=0):
    df = df[df['principle'] > 0].copy()
    month = start_at
    amortization_records = [{
            'month': month,
            'total_payment': 0,
            'interest_paid': 0,
            'principal_paid': 0,
            'remaining_balance': df['principle'].sum()
    }]
    total_principal = df['principle'].sum()
    while total_principal > 0:
        month += 1
        monthly_interest = df['monthly_interest'].sum()
        total_payment = min_payment
        for extra_payment in extra_payments:
            if month >= extra_payment['shift_by'] and (month - extra_payment.get('shift_by', 0)) % extra_payment['num_months'] == 0:
                total_payment += extra_payment['amount']
        principal_payment = max(0, total_payment - monthly_interest)
        if total_principal + monthly_interest < total_payment:
            principal_payment = total_principal
            min_payment = principal_payment + monthly_interest
        total_principal -= principal_payment
        amortization_records.append({
            'month': month,
            'total_payment': total_payment,
            'interest_paid': monthly_interest,
            'principal_paid': principal_payment,
            'remaining_balance': total_principal
        })
        df['payment_proportion'] = df['principle'] / df['principle'].sum()
        df['principle'] -= df['payment_proportion'] * principal_payment
        df['monthly_interest'] = (df['principle'] * df['apr'] / 12 / 100).round(2)
        if total_principal <= 1:
            break
    return pd.DataFrame(amortization_records)

In [35]:
extra_payments = [
    {'num_months': 1, 'amount': 1500, 'shift_by': 0},
    {'num_months': 3, 'amount': net_biannual_bonus, 'shift_by': months_till_next_equity},
    {'num_months': 6, 'amount': net_quarterly_equity, 'shift_by': months_till_next_bonus},
]
amortization_df = amortization_schedule(df_student, min_student_loan_payment, extra_payments) # as of June 1
amortization_df

,month,total_payment,interest_paid,principal_paid,remaining_balance
0,0,0.00,0.00,0.00,23267.76
1,1,1780.22,66.32,1713.90,21553.86
2,2,1780.22,61.44,1718.78,19835.08
3,3,18469.72,56.54,18413.18,1421.90
4,4,1780.22,4.06,1421.90,0.00


In [36]:
num_months_remaining_student = amortization_df['month'].iloc[-1]
print(f'num_months_remaining_student = {num_months_remaining_student}')

num_months_remaining_student = 4


In [37]:
print(f'num_years_remaining_student = {num_months_remaining_student // 12}')
print(f'num_months_leftover_remaining_student = {num_months_remaining_student % 12}')

num_years_remaining_student = 0
num_months_leftover_remaining_student = 4


In [38]:
leftover_student_payment = amortization_df['total_payment'].iloc[-1] - amortization_df['remaining_balance'].iloc[-2]
print(f'leftover_student_payment = ${round(leftover_student_payment, 2)}')

leftover_student_payment = $358.32


## Car Loan

In [39]:
car_payment = 1249.36
car_apr = 1.99

remaining_car_principle = car_principle - num_months_remaining_student * (car_payment / (1 - car_apr / 100)) - leftover_student_payment
df_car = pd.DataFrame(data=[[remaining_car_principle, car_apr]], columns=['principle', 'apr'])
df_car['principle'] = df_car['principle'].round(2)
df_car['monthly_interest'] = (df_car['principle'] * df_car['apr'] / 12 / 100).round(2)
df_car

,principle,apr,monthly_interest
0,26378.29,1.99,43.74


In [40]:
extra_payments = [
    {'num_months': 1, 'amount': 1500, 'shift_by': 0},
    {'num_months': 3, 'amount': net_biannual_bonus, 'shift_by': months_till_next_equity},
    {'num_months': 6, 'amount': net_quarterly_equity, 'shift_by': months_till_next_bonus},
]
amortization_df = amortization_schedule(df_car, car_payment, extra_payments, start_at=num_months_remaining_student)
amortization_df

,month,total_payment,interest_paid,principal_paid,remaining_balance
0,4,0.00,0.00,0.00,26378.29
1,5,2749.36,43.74,2705.62,23672.67
2,6,9187.36,39.26,9148.10,14524.57
3,7,2749.36,24.09,2725.27,11799.30
4,8,2749.36,19.57,2729.79,9069.51
5,9,19438.86,15.04,9069.51,0.00


In [41]:
num_months_remaining_car = amortization_df['month'].iloc[-1]
print(f'num_months_remaining_car = {num_months_remaining_car}')

num_months_remaining_car = 9


In [42]:
print(f'num_years_remaining_car = {num_months_remaining_car // 12}')
print(f'num_leftover_months_remaining_car = {num_months_remaining_car % 12}')

num_years_remaining_car = 0
num_leftover_months_remaining_car = 9


In [43]:
leftover_car_payment = amortization_df['total_payment'].iloc[-1] - amortization_df['remaining_balance'].iloc[-2]
print(f'leftover_car_payment = ${round(leftover_car_payment, 2)}')

leftover_car_payment = $10369.35


## Timeline

In [44]:
num_months_loans = num_months_remaining_student + num_months_remaining_car - num_months_remaining_student
print(f'num_months_loans = {num_months_loans}')

num_months_loans = 9


In [45]:
current_balance = current_savings + leftover_car_payment
monthly_profit_no_loans = avg_effective_monthly_profit + min_student_loan_payment + car_payment - other_monthly_income
print(f'monthly_profit_no_loans = ${round(monthly_profit_no_loans, 2)}')

monthly_profit_no_loans = $8837.75


In [46]:
num_months_to_save = round((savings_target - current_balance) / monthly_profit_no_loans)
print(f'num_months_to_save = {num_months_to_save}')

num_months_to_save = 1


In [47]:
num_months_remaining = num_months_loans + num_months_to_save
print(f'num_months_remaining = {num_months_remaining}')

num_months_remaining = 10


In [48]:
# Student Loans Paid: Dec-Jan 2024
# Car Paid: Apr-May 2025
# Savings Met: Aug-Sep 2025
# depends mainly on how much extra paid per month (which depends on spending)

In [49]:
# import matplotlib.pyplot as plt
# import numpy as np

# L = 2
# h = 1

# x = np.linspace(-L, L, 1000)
# f = lambda x: h * np.where((-L <= x) & (x <= -L/2), 0, np.where((-L/2 < x) & (x < L/2), 1, np.where((L/2 <= x) & (x <= L), 0, np.nan)))

# def partial_sum(x, n):
#     y = h / 2 * np.ones(x.shape)
#     y += sum([2 * h / np.pi * (-1)**n / (2*n + 1) * np.cos((2*n + 1) * np.pi * x / L) for n in range(2*n + 1)])
#     return y

# plt.plot(x, f(x), alpha=1, linewidth=2, zorder=0, label='$f(x)$')
# plt.plot(x, partial_sum(x, 1), alpha=0.8, linewidth=1, zorder=1, label='$S_1$')
# plt.plot(x, partial_sum(x, 5), alpha=0.8, linewidth=1, zorder=2, label='$S_{5}$')
# plt.plot(x, partial_sum(x, 10), alpha=0.8, linewidth=1, zorder=3, label='$S_{10}$')
# plt.plot(x, partial_sum(x, 100), alpha=0.8, linewidth=1, zorder=4, label='$S_{100}$')
# plt.xlabel('$x$')
# plt.ylabel('$y$')
# plt.xlim(-L, L)
# plt.ylim(0, h * 1.2)
# plt.legend(loc='upper right')
# plt.show()

In [50]:
# import matplotlib.pyplot as plt
# import numpy as np
# import scipy.special as sp
# from scipy.integrate import quad
# import math

# f = lambda x: np.where(x < 0, -1, np.where(x > 0, 1, np.nan))
# log_a = lambda n: mpmath.log(2*n + 1) + mpmath.log(mpmath.factorial(n-2)) - mpmath.log(2) * ((n + 1) // 2) - mpmath.log(mpmath.factorial((n-1)//2)) - mpmath.log(mpmath.factorial((n+1)//2))

# def legendre_coefficient(n):
#     integrand_left = lambda x: f(x) * sp.eval_legendre(n, x)
#     integral_left = quad(integrand_left, -1, 0, limit=100)[0]
#     integrand_right = lambda x: f(x) * sp.eval_legendre(n, x)
#     integral_right = quad(integrand_right, 0, 1, limit=100)[0]
#     return (2 * n + 1) / 2 * (integral_left + integral_right)

# def legendre_partial_sum(x, N):
#     y = np.zeros(x.shape)
#     for n in range(N + 1):
#         if n % 2 == 1:
#             an = legendre_coefficient(n)
#             y += an * sp.eval_legendre(n, x)
#     return y

# x = np.linspace(-1, 1, 1000)

# plt.plot(x, f(x), alpha=1, linewidth=2, zorder=0, label='$f(x)$')
# plt.plot(x, legendre_partial_sum(x, 3), alpha=0.8, linewidth=1, zorder=2, label='$S_{3}$')
# plt.plot(x, legendre_partial_sum(x, 5), alpha=0.8, linewidth=1, zorder=3, label='$S_{5}$')
# plt.plot(x, legendre_partial_sum(x, 7), alpha=0.8, linewidth=1, zorder=4, label='$S_{7}$')
# plt.plot(x, legendre_partial_sum(x, 99), alpha=0.8, linewidth=1, zorder=5, label='$S_{99}$')

# plt.xlabel('$x$')
# plt.ylabel('$y$')
# plt.xlim(-1, 1)
# plt.ylim(-2, 2)
# plt.legend(loc='lower right')
# plt.show()